In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive, Button

class DiffusionModel:
    """
    A class to model and plot diffusion through a porous medium.

    This model simulates the transport of a solute through a porous medium, considering both advection 
    (transport due to flow) and diffusion (spreading due to molecular diffusion and mechanical dispersion).

    Attributes:
    ----------
    matrix_porosity : float
        The porosity of the matrix.
    input_conc : float
        The concentration of the input.
    input_duration : float
        The duration of the input.
    time_step : float
        The time step for the simulation.
    t_range_end : float, optional
        The end of the time range for the simulation, default is 250.
    reference_lines : list
        A list to store reference lines for comparison.
    ref_line_params : list
        A list to store parameters of reference lines.
    ref_line_colors : list
        A list to store colors for reference lines.
    """

    def __init__(self, matrix_porosity, input_conc, input_duration, time_step, t_range_end=250):
        """
        Initializes the DiffusionModel class with given parameters.

        Parameters:
        ----------
        matrix_porosity : float
            The porosity of the matrix.
        input_conc : float
            The concentration of the input.
        input_duration : float
            The duration of the input.
        time_step : float
            The time step for the simulation.
        t_range_end : float, optional
            The end of the time range for the simulation, default is 250.
        """
        self.matrix_porosity = matrix_porosity
        self.input_conc = input_conc
        self.input_duration = input_duration
        self.time_step = time_step
        self.t_range_end = t_range_end
        self.reference_lines = []
        self.ref_line_params = []
        self.ref_line_colors = ['black']

    def residence_time(self, length, velocity):
        """
        Calculate the residence time.

        Parameters:
        ----------
        length : float
            The length of the medium.
        velocity : float
            The velocity of the flow.

        Returns:
        -------
        float
            The residence time.
        
        Explanation:
        This function calculates the residence time, which is the time it takes for the solute to travel through 
        the medium based on its length and the flow velocity.
        """
        return length / velocity

    def rel_input_duration(self, input_duration, residence_time):
        """
        Calculate the relative input duration.

        Parameters:
        ----------
        input_duration : float
            The duration of the input.
        residence_time : float
            The residence time.

        Returns:
        -------
        float
            The relative input duration.
        
        Explanation:
        This function calculates the relative input duration as a ratio of the input duration to the residence time.
        """
        return input_duration / residence_time

    def positive_pulse(self, matrix_porosity, pore_diff_coef, length, velocity, a, t_range, residence_t):
        """
        Calculate the positive pulse of the concentration.

        Parameters:
        ----------
        matrix_porosity : float
            The porosity of the matrix.
        pore_diff_coef : float
            The pore diffusion coefficient.
        length : float
            The length of the medium.
        velocity : float
            The velocity of the flow.
        a : float
            A parameter related to the fracture aperture.
        t_range : numpy.ndarray
            The time range array.
        residence_t : float
            The residence time.

        Returns:
        -------
        numpy.ndarray
            The positive pulse concentration array.
        
        Explanation:
        This function calculates the concentration of the solute as it travels through the medium, considering 
        the positive pulse when the solute is introduced. It uses the complementary error function (erfc) to model 
        the diffusion process.
        """
        
        def t_tr(t, residence_time):
            return t / residence_time
        
        c0 = np.zeros(1 * len(t_range))
        i = 0
        
        for ti in t_range:
            if t_tr(ti, residence_t) > 1:
                arg1 = (matrix_porosity * pore_diff_coef * length) / (velocity * a)
                arg2 = np.sqrt(pore_diff_coef * (ti - residence_t))
                c = erfc(arg1 / arg2)
                c0[i] = c
                i += 1
            else:
                c0[i] = 0
                i += 1
                        
        return c0

    def negative_pulse(self, matrix_porosity, pore_diff_coef, length, velocity, a, t, rel_input_dur, input_duration, residence_t):
        """
        Calculate the negative pulse of the concentration.

        Parameters:
        ----------
        matrix_porosity : float
            The porosity of the matrix.
        pore_diff_coef : float
            The pore diffusion coefficient.
        length : float
            The length of the medium.
        velocity : float
            The velocity of the flow.
        a : float
            A parameter related to the fracture aperture.
        t : numpy.ndarray
            The time range array.
        rel_input_dur : float
            The relative input duration.
        input_duration : float
            The duration of the input.
        residence_t : float
            The residence time.

        Returns:
        -------
        numpy.ndarray
            The negative pulse concentration array.
        
        Explanation:
        This function calculates the concentration of the solute after the input has stopped, considering the 
        negative pulse. It uses the complementary error function (erfc) to model the diffusion process.
        """
        
        def t_tr(t, residence_time):
            return t / residence_time
        
        c0 = np.zeros(1 * len(t))
        i = 0
        
        for ti in t:
            if t_tr(ti, residence_t) > 1 + rel_input_dur:
                arg1 = (matrix_porosity * pore_diff_coef * length) / (velocity * a) 
                arg2 = np.sqrt(pore_diff_coef * (ti - residence_t - input_duration))
                c = erfc(arg1 / arg2)
                c0[i] = c
                i += 1
            else:
                c0[i] = 0
                i += 1
                        
        return c0    

    def relative_concentration(self, matrix_porosity, pore_diff_coeff, length, velocity, fracture_aperture, t_range, rel_in_dur, input_duration, res_t):
        """
        Calculate the relative concentration.

        Parameters:
        ----------
        matrix_porosity : float
            The porosity of the matrix.
        pore_diff_coeff : float
            The pore diffusion coefficient.
        length : float
            The length of the medium.
        velocity : float
            The velocity of the flow.
        fracture_aperture : float
            The fracture aperture.
        t_range : numpy.ndarray
            The time range array.
        rel_in_dur : float
            The relative input duration.
        input_duration : float
            The duration of the input.
        res_t : float
            The residence time.

        Returns:
        -------
        numpy.ndarray
            The relative concentration array.
        
        Explanation:
        This function calculates the relative concentration of the solute by subtracting the negative pulse from 
        the positive pulse. This represents the net concentration of the solute in the medium over time.
        """
        # Calculate the positive and negative pulses  
        positiv_p = self.positive_pulse(matrix_porosity, 
                                        pore_diff_coeff,
                                        length,
                                        velocity,
                                        fracture_aperture,
                                        t_range,
                                        res_t)
        
        negative_p = self.negative_pulse(matrix_porosity, 
                                         pore_diff_coeff,
                                         length,
                                         velocity,
                                         fracture_aperture,
                                         t_range,
                                         rel_in_dur,
                                         input_duration,
                                         res_t)
                                         
        # Return the difference between the positive and negative pulses
        return positiv_p - negative_p

    def plot_diffusion(self, matrix_porosity, velocity, fracture_aperture, pore_diff_coeff, length, x_axis_log_scale=False, y_axis_log_scale=False):
        """
        Plot the diffusion model.

        Parameters:
        ----------
        matrix_porosity : float
            The porosity of the matrix.
        velocity : float
            The velocity of the flow.
        fracture_aperture : float
            The fracture aperture.
        pore_diff_coeff : float
            The pore diffusion coefficient.
        length : float
            The length of the medium.
        x_axis_log_scale : bool, optional
            Whether to use a logarithmic scale on the x-axis, default is False.
        y_axis_log_scale : bool, optional
            Whether to use a logarithmic scale on the y-axis, default is False.
        
        Explanation:
        This function plots the relative concentration of the solute over time. It can also include reference lines 
        for comparison. The x and y axes can optionally be set to logarithmic scales.
        """
        
        t_range_end = 250
        self.matrix_porosity = matrix_porosity
        t_range = np.arange(0, t_range_end, self.time_step)
        results = []

        self.length = length

        # Calculations
        residence_t = self.residence_time(self.length, velocity)
        rel_in_dur = self.rel_input_duration(self.input_duration, residence_t)
        rel_conc = self.relative_concentration(self.matrix_porosity,
                                               pore_diff_coeff,
                                               self.length,
                                               velocity,
                                               fracture_aperture,
                                               t_range,
                                               rel_in_dur,
                                               self.input_duration,
                                               residence_t)
        results.append(rel_conc)

        # Plotting
        fig, ax = plt.subplots()
        for res, clr in zip(results, ['blue']):
            ax.plot(t_range, res, color=clr, label='Current: V={:.2e}, L={:.2e}, RT={}, A={:.2e}, Dm={:.2e}'.format(velocity, length, round(residence_t, 2), fracture_aperture, pore_diff_coeff))

        # Plot each reference line with formatted parameters in the legend
        for ((t_range_ref, rel_conc_ref), (vel_ref, length, ap_ref, coeff_ref, residence_time), color) in zip(self.reference_lines, self.ref_line_params, self.ref_line_colors):
            ax.plot(t_range_ref, rel_conc_ref, color=color, label='V={:.2e}, L={:.2e}, RT={}, A={:.2e}, Dm={:.2e}'.format(vel_ref, length, round(residence_time, 2), ap_ref, coeff_ref))

        if x_axis_log_scale:
            ax.set_xscale('log')
        if y_axis_log_scale:
            ax.set_yscale('log')

        ax.set_xlim(0, 30)
        ax.set_ylim(0.00001, 1.1)
        ax.set_xlabel('Time [days]')
        ax.set_ylabel('Relative Concentration [-]')
        ax.legend(loc='upper right', bbox_to_anchor=(2.2, 1.0))
        plt.show()

    def on_button_click(self, button):
        """
        Handle the button click event to add a reference line.

        Parameters:
        ----------
        button : ipywidgets.Button
            The button that was clicked.
        
        Explanation:
        This function is called when the "Add Reference Line" button is clicked. It adds the current model parameters 
        and results as a reference line for comparison in future plots.
        """
        self.add_new_color()

        # Get current slider values
        velocity = self.velocity_slider.value
        fracture_aperture = self.fracture_aperture_slider.value
        pore_diff_coeff = self.pore_diff_coeff_slider.value

        # Use self.t_range_end here
        t_range = np.arange(0, self.t_range_end, self.time_step)
        residence_t = self.residence_time(self.length, velocity)
        rel_in_dur = self.rel_input_duration(self.input_duration, residence_t)
        rel_conc = self.relative_concentration(self.matrix_porosity,
                                               pore_diff_coeff,
                                               self.length,
                                               velocity,
                                               fracture_aperture,
                                               t_range,
                                               rel_in_dur,
                                               self.input_duration,
                                               residence_t)
        
        
        # Store the current line (parameters and results) for future reference
        self.reference_lines.append((t_range, rel_conc))
        self.ref_line_params.append((velocity, self.length, fracture_aperture, pore_diff_coeff, residence_t))

        # Clear output and redraw the plot including reference lines
        clear_output(wait=True)
        self.create_interactive_plot()

    def add_new_color(self):
        """
        Add a new color to the reference lines.
        
        Explanation:
        This function adds a new color to the reference lines to distinguish between different sets of parameters. 
        It cycles through a predefined list of colors and ensures each new reference line has a unique color until all 
        colors are used up.
        """
        colors = ['orange', 'green', 'red', 'purple', 'brown', 'pink']
        for color in colors:
            if color not in self.ref_line_colors:
                self.ref_line_colors.append(color)
                return
        print("All colors used up!")

    def create_interactive_plot(self):
        """
        Create an interactive plot with sliders and buttons.
        
        Explanation:
        This function creates an interactive plot using ipywidgets. It allows users to adjust parameters such as 
        matrix porosity, velocity, fracture aperture, and pore diffusion coefficient using sliders. Users can also 
        add reference lines and toggle logarithmic scales for the x and y axes.
        """
        # Create sliders as class attributes
        # Initialize interactive widgets with FloatLogSlider for scientific notation
        self.length_slider = widgets.FloatSlider(min=0, max=100, step=1, value=10, description='Length:')
        self.velocity_slider = widgets.FloatLogSlider(min=-1, max=1, step=0.1, value=2, description='Velocity:', readout_format='.2e')
        self.matrix_porosity_slider = widgets.FloatLogSlider(min=-4, max=-0.3, step=0.01, value=0.03, description='Matrix Porosity:', readout_format='.2e')
        self.fracture_aperture_slider = widgets.FloatLogSlider(min=-5, max=-2, step=0.1, value=1e-4, description='Fracture Aperture:', readout_format='.2e')
        self.pore_diff_coeff_slider = widgets.FloatLogSlider(min=-8, max=-6, step=0.1, value=2.59e-7, description='Pore Diff Coeff:', readout_format='.2e')

        # add fields which when activated will plot the axes in log scale
        self.log_scale_x = widgets.Checkbox(value=False, description='Log Scale X')
        self.log_scale_y = widgets.Checkbox(value=False, description='Log Scale Y')

        interactive_plot = interactive(self.plot_diffusion, 
                                        matrix_porosity=self.matrix_porosity_slider,
                                        velocity=self.velocity_slider, 
                                        fracture_aperture=self.fracture_aperture_slider, 
                                        pore_diff_coeff=self.pore_diff_coeff_slider,
                                        length=self.length_slider,
                                        x_axis_log_scale=self.log_scale_x,
                                        y_axis_log_scale=self.log_scale_y)

        # Button for adding reference lines
        button = Button(description="Add Reference Line")
        button.on_click(lambda b: self.on_button_click(b))

        display(interactive_plot, button)


# Example usage
model = DiffusionModel(matrix_porosity=3e-2, input_conc=1, input_duration=2, time_step=0.1)
model.create_interactive_plot()


interactive(children=(FloatLogSlider(value=0.03, description='Matrix Porosity:', max=-0.3, min=-4.0, readout_f…

Button(description='Add Reference Line', style=ButtonStyle())